In [7]:
%load_ext autoreload
%autoreload 1

import json
from pathlib import Path
from tqdm.notebook import tqdm
import time
import pandas as pd
from collections import Counter
import argparse

import sys
sys.path.insert(0, "/data/tir/projects/tir6/strubell/jmilbaue/HTMAX/ddd-benchmark/src") ### PATH TO THIS NOTEBOOK, YOU MUST EDIT THIS!

%aimport llm
from llm import OpenAI
%aimport prompts
from prompts import prompt_db
%aimport prompting
from prompting import build_prompt
%aimport utils
from utils import convert_temperature

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
time_unit_key = 'Time unit [D/m/h]'
temp_unit_key = 'Temp unit [C/K/F]'

# Names of properties in the prompt
properties = [
    'Silicon',
    'Germanium',
    'Aluminum',
    'OH',
    'H2O',
    'HF',
    'SDA ratio',
    'Boron',
    'Time',
    time_unit_key,
    'Temp',
    temp_unit_key,
    'Extracted product',
    'SDA Type'
]

# Remapping some of the names for the reference dataset
map = {
    "Silicon": "Si",
    "Germanium": "Ge",
    "Aluminum": "Al",
    "Boron": "B",
    "Extracted product": "Extracted",
    "SDA ratio": "SDA",
}

In [17]:
def convert_df(df):
    new_rows = []
    for i, row in df.iterrows():
        print(row)
        new_row = {}
        for k in row.keys():
            if k in map:
                new_row[map[k]] = row[k]
            else:
                new_row[k] = row[k]

        time_unit = None
        if time_unit_key in row:
            if isinstance(row[time_unit_key], str):
                time_unit = row[time_unit_key].lower()

        time_value = None
        if 'Time' in row:
            if isinstance(row['Time'], str):
                try:
                    time_value = float(row['Time'])
                except:
                    time_value = None
            elif isinstance(row['Time'], float):
                time_value = row['Time']

        if time_unit is None:
            time_unit = 'h'
        
        new_time = None
        if time_value is not None:
            if 'd' in time_unit:
                time_dilation = 24
            if 'h' in time_unit:
                time_dilation = 1
            if 'm' in time_unit:
                time_dilation = 1 / 60
            new_time = time_value * time_dilation
        
        temp_unit = None
        if temp_unit_key in row:
            if isinstance(row[temp_unit_key], str):
                temp_unit = row[temp_unit_key].lower()

        temp_value = None
        if 'Temp' in row:
            if isinstance(row['Temp'], str):
                try:
                    temp_value = float(row['Temp'])
                except:
                    temp_value = None
            elif isinstance(row['Temp'], float):
                temp_value = row['Temp']

        if temp_unit is None:
            temp_unit = "C"

        new_temp = None
        if temp_value is not None:
            new_temp = convert_temperature(float(row["Temp"]), "C" if "c" in temp_unit else ("F" if "f" in temp_unit else "K"), "C")

        new_row["Time"] = new_time
        new_row["Temp"] = new_temp
        new_rows.append(new_row)

    new_df = pd.DataFrame(new_rows)[["Si", "Ge", "Al", "OH", "H2O", "HF", "SDA", "B", "Time", "Temp", "SDA Type", "Extracted", "source", "doc_processing_time", "total_response_chars"]]
    return new_df

def get_responses(model, modal, author, keyfile, indir, ext):
    keydata = json.load(open(keyfile))
    oai_org = keydata['openai_org']
    oai_project = keydata['openai_project']

    if "gpt" in model:
        llm = OpenAI("gpt-4o", oai_org, oai_project)

    if ext == "png":
        prompt = build_prompt(model, modal, author, properties, prompt_db)

    for f in tqdm(sorted(indir.glob(f"*{ext}")), desc="Processing documents", position=0):
        if ext == "ml":
            with open(f) as fin:
                xml_data = fin.read()
                prompt = build_prompt(model, modal, author, properties, prompt_db, context=xml_data)

        try:
            start = time.time()
            pbar = tqdm(desc="Streaming response characters", position=1)
            if ext == 'png':
                response = llm(prompt, image_path=str(f), use_json=True, pbar=pbar)
            else:
                response = llm(prompt, use_json=True, pbar=pbar)
            stop = time.time()
            response['source'] = str(f.name)
            yield {'response': response, 'time': stop-start}
        except Exception as e:
            print(f"ERROR for {f}")
            print(e)


In [ ]:
model = "gpt4o"
modal = "visual"
author = "hungyi"
ext = "png"
indir = Path("../data/imgs/")
outfile = Path(f"../results/{model}_{modal}_{author}.test.csv")

clean_recipes = []
for x in get_responses(
    model,
    modal,
    author,
    keyfile=Path("../secrets.json"),
    indir=indir,
    ext=ext,
):
    response = x['response']
    timing = x['time']
    if 'recipes' in response:
            for recipe in response['recipes']:
                recipe_template = {k:None for k in properties}
                for k in recipe:
                    if k in properties:
                        recipe_template[k] = recipe[k]
                recipe_template['source'] = response['source']
                recipe_template['doc_processing_time'] = timing
                recipe_template['total_response_chars'] = len(json.dumps(response))
                clean_recipes.append(recipe_template)

df = pd.DataFrame(clean_recipes)
df.to_csv(f"{outfile}.tmp")

clean = convert_df(df)
clean.to_csv(outfile)

Processing each document: 0it [00:00, ?it/s]

Processing documents:   0%|          | 0/3 [00:00<?, ?it/s]

Streaming response characters: 0it [00:00, ?it/s]

NameError: name 't' is not defined